In [1]:
import tensorflow as tf

In [4]:
def parse_tfrecord(file):
    feature_description = {
        'note': tf.io.FixedLenFeature([], tf.int64),
        'note_str': tf.io.FixedLenFeature([], tf.string),
        'instrument': tf.io.FixedLenFeature([], tf.int64),
        'instrument_str': tf.io.FixedLenFeature([], tf.string),
        'pitch': tf.io.FixedLenFeature([], tf.int64),
        'velocity': tf.io.FixedLenFeature([], tf.int64),
        'sample_rate': tf.io.FixedLenFeature([], tf.int64),
        'audio': tf.io.FixedLenFeature([64000], tf.float32), # 4 seconds at 16kHz
        'qualities': tf.io.FixedLenFeature([10], tf.int64),
        'qualities_str': tf.io.FixedLenFeature([], tf.string),
        'instrument_family': tf.io.FixedLenFeature([], tf.int64),
        'instrument_family_str': tf.io.FixedLenFeature([], tf.string),
        'instrument_source': tf.io.FixedLenFeature([], tf.int64),
        'instrument_source_str': tf.io.FixedLenFeature([], tf.string)
    }
    
    return tf.io.parse_single_example(file, feature_description)